# Concepts

## General idea

We take following ideas from:
* `Isolation Forest` - To identify outliers perform random splits in the attribute space. Anomalies will be isolated very soon thanks to it and we can easily identify them by checking average depth.
* `Similarity Forest` - This was created with classification task in mind. It can work with any type of objects and is based on similarity between them. So having our dataset with 4 examples and 3 attributes:

In [1]:
import numpy as np
np.random.seed(23)
X = np.array([
    [1, 4, 2],
    [2, 3, 1],
    [3, 2, 4],
    [4, 1, 3],
])
print(X)

[[1 4 2]
 [2 3 1]
 [3 2 4]
 [4 1 3]]


We at first select a pair of objects $O_{i}$ and $O_{j}$. In this case this is object 0 and 3.

In [2]:
i, j = 0, 3
Oi, Oj = X[i], X[j]
print(Oi, Oj)

[1 4 2] [4 1 3]


Finally we perform a projection of every object $k$ based on similarity $S_{ki}$ and $S_{kj}$ defined as:

$$
    Projection(k) = S_{ki} - S_{kj}
$$

e.g we can define this projection w.r.t euclidean dot product as a similarity measure

$$
    Projection(k) = dot(O_{k},O_{i}) - dot(O_{k}, O_{j})
$$

In our example we obtain

In [3]:
dot_oxi = np.dot(X, Oi)
dot_oxj = np.dot(X, Oj)
print("Dot(X, Oi) = ", dot_oxi)
print("Dot(X, Oj) = ", dot_oxj)
print("Projection = ", dot_oxi - dot_oxj)

Dot(X, Oi) =  [21 16 19 14]
Dot(X, Oj) =  [14 14 26 26]
Projection =  [  7   2  -7 -12]


After that, we perform everything as in the Isolation Forest algorithm. Every time a different $O_{i}$ and $O_{j}$ is selected we obtain different projection. The example may look very simple. However, this approach is quite powerful as object and this similarity functions can be literally of any type which was tested in our third inspiration.
* `Random Similarity Forest` - here authors extended Similarity Forest to work with objects like graphs, distributions, sets etc. and moreover allowed using of mixed datasets. Imagine one column of your data is a graph and another is a set. You can easily use this algorithm by just defining some similarity measures between them.


In `Generalized Isolation Forest (GIF)`we merge all this ideas and add some small tweaks to allow easily use any of the proposed solutions above.


## Data object

As said previously GIF can work with mixed datasetypes - one instance in a dataset can be represented by many modalities at the same type. Moreover you can assign as many distance functions to every modality as you want you can visualize it like this.

![fishy](_static/dataset.png)

Now:
1. *F1* is a nominal feature you can use `Dice Coefficiant` for it.
2. *F2* is a numerical vector that in general can be in $\mathbb{R}^n$. You can use Euclidean, Manhattan, any p-norm or cosine similarity.
3. *F3* Here we have distribution so you can use Kolmogorov-Smirnov distance, Information-divergence
4. *F7* with graph you can go with Ipsen-Mikailov, Portrait etc.

Some distances are computationally heavy and using them on entire data would be prohibitive that's why we introduce distance `precomputing` and `n_selected_objects` parameter that restrict number of necessary computations. We write about them later.

So how to create this kind of dataset? We introduce `RISFData` class

In [42]:
%load_ext autoreload
%autoreload 2

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))
class Graph():
    pass

def jaccard_projection(x, y):
    pass

def euclidean_projection(x, y):
    pass

def cosine_projection(x, y):
    pass

def jaccard_distance(x, y):
    pass

def ipsen_mikailov(x, y):
    pass

def portrait(x, y):
    pass

class Tree():
    pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from rsif.rsif_data import RsifData
from rsif.distance import TrainDistanceMixin, SelectiveDistance # We will explain them later
data = RsifData()

categorical_feature = np.array([[0], [2], [1]])
distances_categorical = [SelectiveDistance(projection_func = jaccard_projection, min_n=1, max_n=1)]
data.add_data(categorical_feature, distances_categorical)

numerical_features = np.random.randn(3, 3) # vector in R^3
distances_numerical = [SelectiveDistance(projection_func = euclidean_projection, min_n=1, max_n=3),
                        SelectiveDistance(projection_func = cosine_projection, min_n=1, max_n=3)]
data.add_data(numerical_features, distances_numerical)

set_features = np.array([{10, 20, 30}, {20, 30}, {10, 20}])
distances_set = [TrainDistanceMixin(distance = jaccard_distance)]
data.add_data(set_features, distances_set)

graph_features = np.array([Graph(), Graph(), Graph()]) # Graph is just a dummy object
distances_graph = [TrainDistanceMixin(distance = ipsen_mikailov), TrainDistanceMixin(distance = portrait)]
data.add_data(graph_features, distances_graph)

I know it's a little bit of writing, but don't worry if you have just a simple numerical data you can just go with

In [31]:
from rsif.forest import RandomSimilarityIsolationForest

data = np.random.randn(100,10)
forest = RandomSimilarityIsolationForest().fit(data)

Coming back to our example. `RsifData` inheriths from a list and add few additional features. So if you want you can easily iterate over previously added data and distance functions

In [21]:
for i, feature in enumerate(data):
    print(feature, feature.dtype, data.distances[i])

[[0]
 [2]
 [1]] int32 [<risf.distance.TrainDistanceMixin object at 0x0000015FD2FCEAA0>]
[[1.77 -0.347 0.67]
 [0.322 0.0603 -1.04]
 [-1.01 0.442 1.13]] float64 [<risf.distance.TrainDistanceMixin object at 0x0000015FD2FCDAE0>, <risf.distance.TrainDistanceMixin object at 0x0000015FD2FCDB40>]
[{10, 20, 30} {20, 30} {10, 20}] object [<risf.distance.TrainDistanceMixin object at 0x0000015FD2FCCA60>]
[<__main__.Graph object at 0x0000015FD2FCDC30> <__main__.Graph object at 0x0000015FD2FCDD50> <__main__.Graph object at 0x0000015FD2FCF430>] object [<risf.distance.TrainDistanceMixin object at 0x0000015FD2FCE2F0>, <risf.distance.TrainDistanceMixin object at 0x0000015FD2FCF3D0>]


Now to fit the model you would just do 

```python
forest = RandomIsolationSimilarityForest(distances = data.distances).fit(data)
```

Very important step hapenning inside fit function is parsing this Rsif Data container into plain numpy array

In [18]:
from rsif.utils.validation import prepare_X

parsed_data, features_span = prepare_X(data)

In [23]:
parsed_data, features_span

(array([[0, 1.77, -0.347, 0.67, 0, 0],
        [2, 0.322, 0.0603, -1.04, 1, 1],
        [1, -1.01, 0.442, 1.13, 2, 2]]),
 [(0, 1), (1, 4), (4, 5), (5, 6)])

As you can see data was merged into one big table. Moreover every feature got it's own feature span. You may wonder why instead of sets and graphs objects we have this indices. for internal operations of the algorithm to make it much faster we replace object with it's indices. This will become easier after we describe `SelectiveDistance` and `TrainDistanceMixin`.

But just to finish this part fit procedure at every split performs such operations:
1. Get non unique features -> if there are 0 set_node to be a leaf
2. Select feature randomly
3. Select one of the available distances randomly
4. project data w.r.t selected feature
5. randomly select split point

## Distances

We have two types of distances objects:
* `SelectiveDistance` - use it when your projection function is very cheap to calculate (Euclidean, Cosine etc.)
* `TrainDistanceMixin` - use it for complex objects when distance computation can take substantial amount of time.


SelectiveDistance can be created as follows

In [65]:
from rsif.distance_functions import euclidean_projection
random_number_generator = np.random.RandomState(23)

X = np.random.randn(10, 10)
Oi = X[0]
Oj = X[1]
distance = SelectiveDistance(projection_func = euclidean_projection, min_n=1, max_n=5)

the question is why we need this min_n and max_n. The two parameters balances between simply Isolation Forest behaviour - selecting individual attributes or Similarity Forest - taking into account entire objects. With setup as above size of selected vector is chosen randomly at every step and we can use at most 5 features at a time. So this setup is closer to the Isolation Forest

In [66]:
tree = Tree()
result = distance.project(X, Oi, Oj, tree, random_number_generator) # we pass random number generator to make the result reproducible
print(result)
tree.selected_features

[2.3 -4.03 0.893 -0.272 4.41 0.215 -1.59 1.37 -2.08 2.67]


array([5, 7, 2, 3])

In [64]:
tree1 = Tree()
result = distance.project(X, Oi, Oj, tree1, random_number_generator) # we pass random number generator to make the result reproducible
print(result)
tree1.selected_features

[-0.0374 -2.6 -1.58 2.47 -1.83 0.657 -0.39 2.1 -1.58 -1.4]


array([2])

So as you can see this is quite powerful mechanism. By setting $min_n$ and $max_n$ to $1$ we obtain `Isolation Forest` like behaviour. By setting $min_n$ and $max_n$ to $n_dim$ we obtain `Similarity Forest` like behaviour. Additionaly, we can explore everything in between.

Train distance mixin basically allows you to precompute distance matrix once and reuse it anytime. It is very useful when distance calculation is expensive e.g for graph distances.

To present inner working of this class I will use unit tests :) 

In [72]:
from rsif.distance import TrainDistanceMixin

class MockDist:
    def __init__(self):
        self.i = 0

    def __call__(self, x, y):
        self.i += 1
        return self.i
    
N_TRAIN_OBJECTS = 5
X = np.empty(N_TRAIN_OBJECTS, dtype=object)
X[:] = object()

In [78]:
distance = TrainDistanceMixin(distance=MockDist())
distance.precompute_distances(X, n_jobs=1)
distance.distance_matrix

array([[0, 1, 2, 3, 4],
       [1, 0, 5, 6, 7],
       [2, 5, 0, 8, 9],
       [3, 6, 8, 0, 10],
       [4, 7, 9, 10, 0]])

So as you can see we have our distances precomputed. Now our Trees can use it. But what if computations are extremely heavy? We can restrict number of available objects for pairs creations, we've empirically shown that this doesn't deteriorate performance of the algorithm.

In [79]:
distance = TrainDistanceMixin(distance=MockDist())
distance.selected_objects = [2, 4]
distance.precompute_distances(X, n_jobs = 1)
distance.distance_matrix

array([[0, 0, 1, 0, 5],
       [0, 0, 2, 0, 6],
       [1, 2, 0, 3, 4],
       [0, 0, 3, 0, 7],
       [5, 6, 4, 7, 0]])

Now during fitting our forest it will be prohibited to use different objects than 2nd and 4th for this particular feature. If you remember RsifData you can set n_selected_objects to some integer which will automatically perform all the stuff for you

```python
data = RsifData()
```

Moreover you don't need to run this precompute_distances manually, as you created RsifData object just run

```python
data.precompute_distances()
```

And finally
```python
forest.fit(data)
```

These are basically all concepts that should be important to you during testing this solution. Now, sky is the limit :)